In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import pickle
from sklearn.preprocessing import MinMaxScaler

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
regenerate_context_data = False
sequence_augmentation = True
whole_session_context = False
model_test_run = False
data_autoencoder = False
pad_to_window_size = True

PATH_TO_LOAD = '../data/04_Merged'
combined_context_path = '../data/05_Interaction_Sequences/context.csv'

window = 100 #seconds

base_path = '../datasets/sequential/'
augmentation_folder = 'aug/' if sequence_augmentation else 'non_aug/'
if model_test_run:
    augmentation_folder = 'test/aug/' if sequence_augmentation else 'test/non_aug/'

sequence_context_path = f'{base_path}{augmentation_folder}parameters/sequence_context.csv'
parameter_path = f'{base_path}{augmentation_folder}parameters/param.pkl'
train_session_path = f'{base_path}{augmentation_folder}parameters/train_sessions.pkl'
test_session_path = f'{base_path}{augmentation_folder}parameters/test_sessions.pkl'
train_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/train.csv'
test_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/test.csv'
train_static_context_path = f'{base_path}{augmentation_folder}static_context/train.csv'
test_static_context_path = f'{base_path}{augmentation_folder}static_context/test.csv'

In [3]:
all_columns = ['index', 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'odometer', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'ESP_speed', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l', 'seatbelt_rear_m',
       'seatbelt_rear_r', 'CHA_ESP_drive_mode', 'CHA_MO_drive_mode',
       'rain_sensor', 'street_category', 'kickdown', 'altitude',
       'driving_program', 'datetime', 'session', 'Label', 'ID',
       'FunctionValue', 'domain', 'BeginTime', 'time_second',
       'distance_driven', 'ts_normalized', 'weekday']

selected = [ 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l',
       'seatbelt_rear_r', 'street_category', 'altitude',
       'datetime', 'session', 'time_second',
       'distance_driven', 'weekday'
]

bad_quality = ['CHA_ESP_drive_mode', 
             'CHA_MO_drive_mode',
             'rain_sensor',
             'kickdown',
             'ESP_speed',
             'seatbelt_rear_m',
            'driving_program',
            'ts_normalized'
             ]

dynamic_context_var = ['avg_irradiation', 'steering_speed', 'temperature_out', 
                       'light_sensor_rear', 'light_sensor_front', 
                       'temperature_in', 'KBI_speed', 'soc', 'latitude',
                       'longitude',  'street_category', 'altitude','time_second',
                       'distance_driven']
static_context_var = ['car_id', 'month', 'weekday', 'hour', 'seatbelt_codriver', 'seatbelt_rear_l',
                       'seatbelt_rear_r',]

In [4]:
def load_context(vehicle):
    context_lists = dynamic_context_var + static_context_var + ['session', 'datetime']
    context_lists.remove('car_id')
    df = pd.read_csv(os.path.join(PATH_TO_LOAD, vehicle + "_merged.csv"), parse_dates=['datetime']
    # df_filt = df[context_lists]
    # df_filt = df_filt.dropna(subset=['KBI_speed'])
    df = df.sort_values(by=['session','datetime'])
    return df

vehicles = ['SEB880','SEB882','SEB883','SEB885','SEB888','SEB889']
context_data_list = []
if regenerate_context_data == True:
    for vehicle in tqdm(vehicles):
        context_curr = load_context(vehicle)
        context_curr['car_id'] = vehicle
        context_data_list.append(context_curr)
    context_data = pd.concat(context_data_list, axis=0)
    context_data.to_csv(combined_context_path)
else:
    context_data = pd.read_csv(combined_context_path, parse_dates=['datetime'], index_col=0)

In [5]:
vehicle_list = context_data.car_id.unique().tolist()
random.shuffle(vehicle_list)
vehicle_dict = {}
used_values = set()
for vehicle in vehicle_list:
    value = random.randint(1, len(vehicle_list))
    while value in used_values:
        value = random.randint(1, len(vehicle_list))
    vehicle_dict[vehicle] = value
    used_values.add(value)

context_data['car_id'] = context_data['car_id'].map(vehicle_dict)
context_data = context_data.sort_values(by=['session','datetime'])

context_data['session'] = context_data['session'].astype(int)
# dynamic_context_var.extend(['session', 'datetime'])
context_data = context_data[dynamic_context_var + static_context_var + ['session', 'datetime']]
# context_var = [item for item in dynamic_context_var + static_context_var if item not in ['session', 'datetime']]
context_data = context_data.groupby(['session', 'datetime'])[dynamic_context_var + static_context_var].mean().reset_index()

with open(train_session_path, 'rb') as pickle_file:
    train_sessions = pickle.load(pickle_file)
with open(test_session_path, 'rb') as pickle_file:
    test_sessions = pickle.load(pickle_file)
    # additional_columns = ['session_id']
context_data = context_data[context_data['session'].isin(train_sessions + test_sessions)]

selected_sequence = pd.read_csv(sequence_context_path, parse_dates=['datetime'], index_col=0).reset_index()
selected_sequence['session'] = selected_sequence['session'].astype(int)
min_datetime_indices = selected_sequence.groupby('session')['datetime'].idxmin()
selected_sequence = selected_sequence.drop(min_datetime_indices)
selected_sequence.reset_index(drop=True, inplace=True)
# context_data = context_data[context_data['session'].isin(selected_sequence.session.unique().tolist())]
selected_dfs = []
for session in tqdm(selected_sequence['session'].unique().tolist()):
    selected_sequence_curr = selected_sequence[selected_sequence['session']==session]
    context_data_curr = context_data[context_data['session']==session]
    context_data_curr = context_data_curr[context_data_curr['datetime']<=selected_sequence_curr['datetime'].max()]
    selected_dfs.append(context_data_curr)
training_sequence_context = pd.concat(selected_dfs, axis=0)

100%|██████████| 1203/1203 [00:10<00:00, 111.34it/s]


In [6]:
# testing_sessions = [16, 25]
# selected_sequence = selected_sequence[selected_sequence['session'].isin(testing_sessions)]
# training_sequence_context = training_sequence_context[training_sequence_context['session'].isin(testing_sessions)]

window_id = 0
if sequence_augmentation == True:
    grouped_selected_sequence = selected_sequence.groupby('session')
    augmented_frames = []
    for session, selected_sequence_curr in tqdm(grouped_selected_sequence):
        for i, row in selected_sequence_curr.iloc[::-1].iterrows():
            context_filt_curr = training_sequence_context[
                (training_sequence_context['session'] == session) &
                (training_sequence_context['datetime'] <= row['datetime'])
            ].copy()
            if not whole_session_context:
                context_filt_curr = context_filt_curr.tail(window)
            context_filt_curr['window_id'] = window_id
            # context_filt_curr['session'] = session
            augmented_frames.append(context_filt_curr)
            window_id += 1
    training_sequence_context_augmented = pd.concat(augmented_frames, axis=0)
    context_data = training_sequence_context_augmented.reset_index(drop=True)
else:
    # if sequence_augmentation is set to false
    if not whole_session_context:
        context_data = training_sequence_context.groupby('session').tail(window)
    context_data = training_sequence_context.reset_index(drop=True)
    context_data['window_id'] = context_data.groupby('session').ngroup()

100%|██████████| 1203/1203 [01:50<00:00, 10.89it/s]


In [7]:
dynamic_context = context_data[dynamic_context_var + ['window_id', 'session', 'datetime']]
print('number of dynamic context session', len(dynamic_context.window_id.unique().tolist()))

# function to pad first value to fit the window size
if pad_to_window_size:
    df = dynamic_context.copy()
    session_counts = df.groupby('window_id').size()
    less_than_100 = session_counts[session_counts < window].index.tolist()
    print(f'Number of window with window length less than {window}: ', len(less_than_100))
    window100_dfs = df[~df['window_id'].isin(less_than_100)]
    empty_df = []
    for window_id in tqdm(less_than_100):
        sub_df = df[df['window_id'] == window_id]
        rows_to_pad = window - len(sub_df)
        min_datetime_row = sub_df.loc[sub_df['datetime'].idxmin()]
        pad_df = pd.DataFrame(min_datetime_row, df.columns).transpose()
        pad_df = pd.concat([pad_df] * int(rows_to_pad), ignore_index=True, axis=0)

        padded_df = pd.concat([pad_df, sub_df], axis=0).reset_index(drop=True)
        padded_df['window_id'] = window_id
        empty_df.append(padded_df)
    if empty_df:
        df = pd.concat(empty_df, axis=0).reset_index(drop=True)
        df = pd.concat([df, window100_dfs], axis=0).sort_values(by=['window_id']).reset_index(drop=True)

        session_counts = df.groupby('window_id').size()
        less_than_100 = session_counts[session_counts < window].index.tolist()
        print(f'Number of window with window length less than {window}: ', len(less_than_100))
        dynamic_context = df

static_context = context_data[static_context_var + ['window_id', 'session', 'datetime']]
print('number of windows', len(dynamic_context.window_id.unique().tolist()), len(static_context.window_id.unique().tolist()))
print('number of session', len(dynamic_context.session.unique().tolist()), len(static_context.session.unique().tolist()))

number of dynamic context session 5454
Number of window with window length less than 100:  157


100%|██████████| 157/157 [00:01<00:00, 153.47it/s]


Number of window with window length less than 100:  0
number of windows 5454 5454
number of session 1203 1203


In [8]:
static_context_list = []
grouped_static_context = static_context.groupby('window_id')

for window, static_context_curr in tqdm(grouped_static_context):
    unique_curr = static_context_curr.drop_duplicates(subset=static_context_curr.columns.difference(['datetime']))
    if len(unique_curr) > 1:
        most_repeated_values = static_context_curr.mode().iloc[0]
        result_df = pd.DataFrame(most_repeated_values).transpose()
        result_df = result_df.reset_index(drop=True)
        static_context_list.append(result_df)
    else:
        static_context_list.append(unique_curr)

if static_context_list:
    static_context = pd.concat(static_context_list, axis=0).reset_index(drop=True)

train_dynamic_context = dynamic_context[dynamic_context['session'].isin(train_sessions)]
test_dynamic_context = dynamic_context[dynamic_context['session'].isin(test_sessions)]
train_static_context = static_context[static_context['session'].isin(train_sessions)]
test_static_context = static_context[static_context['session'].isin(test_sessions)]

if data_autoencoder:
    train_dynamic_context.to_csv('../pretrain/time-series-autoencoder/data/dynamic_context_train.csv', index=False)
    test_dynamic_context.to_csv('../pretrain/time-series-autoencoder/data/dynamic_context_test.csv', index=False)

print('number of session', len(train_dynamic_context.window_id.unique().tolist()), len(test_dynamic_context.window_id.unique().tolist()),
       len(train_static_context.window_id.unique().tolist()), len(test_static_context.window_id.unique().tolist()))

100%|██████████| 5454/5454 [00:13<00:00, 415.16it/s]


number of session 4426 1028 4426 1028


In [9]:
train_static_context.columns

Index(['car_id', 'month', 'weekday', 'hour', 'seatbelt_codriver',
       'seatbelt_rear_l', 'seatbelt_rear_r', 'window_id', 'session',
       'datetime'],
      dtype='object')

In [10]:
train_dynamic_context.columns

Index(['avg_irradiation', 'steering_speed', 'temperature_out',
       'light_sensor_rear', 'light_sensor_front', 'temperature_in',
       'KBI_speed', 'soc', 'latitude', 'longitude', 'street_category',
       'altitude', 'time_second', 'distance_driven', 'window_id', 'session',
       'datetime'],
      dtype='object')

In [11]:
#Normalisation
dynamic_context_to_normalize = [col for col in train_dynamic_context.columns if col not in ['window_id', 'session_ids', 'datetime', 'session_id', 'session']]
scaler_dynamic_context = MinMaxScaler()
scaler_dynamic_context.fit(train_dynamic_context[dynamic_context_to_normalize])
train_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(train_dynamic_context[dynamic_context_to_normalize])
test_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(test_dynamic_context[dynamic_context_to_normalize])

train_dynamic_context.to_csv(train_dynamic_context_path, index=False)
test_dynamic_context.to_csv(test_dynamic_context_path, index=False)

train_static_context.to_csv(train_static_context_path, index=False)
test_static_context.to_csv(test_static_context_path, index=False)

/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_20177/2682103817.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(train_dynamic_context[dynamic_context_to_normalize])
/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_20177/2682103817.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(test_dynamic_context[dynamic_c

In [12]:
train_dynamic_context

,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,soc,latitude,longitude,street_category,altitude,time_second,distance_driven,window_id,session,datetime
0,0.00,0.0,0.666667,0.027668,0.122429,0.663462,0.0,0.571357,0.930601,0.675377,0.142857,0.393889,0.066776,0.256520,0,16,2022-09-09 18:41:18
1,0.00,0.0,0.666667,0.031621,0.133203,0.663462,0.0,0.571357,0.930601,0.675379,1.000000,0.393115,0.068474,0.256520,0,16,2022-09-09 18:42:39
2,0.00,0.0,0.666667,0.031621,0.133203,0.663462,0.0,0.571357,0.930601,0.675379,1.000000,0.393115,0.068453,0.256520,0,16,2022-09-09 18:42:38
3,0.00,0.0,0.666667,0.031621,0.133203,0.663462,0.0,0.571357,0.930601,0.675379,1.000000,0.393115,0.068432,0.256520,0,16,2022-09-09 18:42:37
4,0.00,0.0,0.666667,0.031621,0.132943,0.663462,0.0,0.571357,0.930601,0.675379,1.000000,0.393115,0.068390,0.256520,0,16,2022-09-09 18:42:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545195,0.01,0.0,0.433333,0.027668,0.509408,0.661372,0.0,0.594049,0.935801,0.679297,0.714286,0.341032,0.128855,0.002931,5451,5389,2023-03-28 18:37:19
545196,0.01,0.0,0.433333,0.027668,0.504374,0.660746,0.0,0.593545,0.935801,0.679297,0.000000,0.341032,0.128834,0.002931,5451,5389,2023-03-28 18:37:18
545197,0.01,0.0,0.433333,0.027668,0.503357,0.655942,0.0,0.593545,0.935801,0.679297,0.000000,0.341032,0.128813,0.002931,5451,5389,2023-03-28 18:37:17
545198,0.01,0.0,0.433333,0.027668,0.503428,0.653846,0.0,0.593041,0.935801,0.679297,1.000000,0.341032,0.128792,0.002931,5451,5389,2023-03-28 18:37:16


In [13]:
print(len(train_static_context.session.unique().tolist()), len(test_static_context.session.unique().tolist()))
print(len(train_dynamic_context.window_id.unique().tolist()), len(test_dynamic_context.window_id.unique().tolist()))

963 240
4426 1028
